In [ ]:
!pip install --upgrade pandas
import pandas as pd
import numpy as np
pd.__version__


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd '/content/drive/My Drive/Practicas/DB'
path='/content/drive/My Drive/Practicas/DB/{}'

In [ ]:
!ls

In [ ]:
pd.options.display.float_format='{:,.2f}'.format

In [ ]:
df=pd.read_csv(path.format('Adidas_final.csv'),sep=';')

df.head(5)

In [ ]:
def eliminar_cantidad_valores_unicos (df,x):
    lista=[]
    for i in df.columns:
      if len(list(df[i].unique())) == x:       
        lista.append(i)   
    return (df.drop(lista,axis=1))

In [ ]:
df_limpio=eliminar_cantidad_valores_unicos(df,1)
df_limpio

In [ ]:
df_limpio=df_limpio.convert_dtypes()

In [ ]:
df_limpio['Last Visited']=pd.to_datetime(df_limpio['Last Visited'],errors='coerce',format = '%Y/%m/%d %H:%M:%S')

In [ ]:
df_limpio.dtypes

In [ ]:
df_limpio

In [ ]:
class Transformacion_DataFrame:
  '''Importa el csv y lo transforma'''
  def __init__(self):   
       self.__df=pd.read_csv('Adidas_final.csv',sep=';')
       self.__eliminar_cantidad_valores_unicos()
       self.__cambiar_Types()
       self.__eliminar_URLs()
       self.__arreglar_nombres_columnas()

  '''Elimina filas sin valores relevantes'''
  def __eliminar_cantidad_valores_unicos (self,x=1):    
    try:
      lista=[]
      for i in self.__df.columns:
        if len(list(self.__df[i].unique())) == x:       
          lista.append(i)   
      self.__df=self.__df.drop(lista,axis=1)
    except:
      print('Error al buscar y eliminar filas sin contenido relevante')

  '''Establece trypes de las columnas al formato correcto'''
  def __cambiar_Types(self):
     try: 
      self.__df=self.__df.convert_dtypes()
      self.__df['Last Visited']=pd.to_datetime(self.__df['Last Visited'],errors='coerce',format = '%Y/%m/%d %H:%M:%S')
     except:
       print('Error al convertir los tipos')

  def __eliminar_URLs(self):
    '''Elimina URLs y direcciones'''
    try:
      self.__df=self.__df.drop(['URL','Images'],axis=1)
    except:
      print('Error al borrar URLs')

  '''Reemplaza espacios por giones'''
  def __arreglar_nombres_columnas(self):
    try:
      lista_columnas=[]
      lista_columnas=self.__df.columns
      for x in lista_columnas:
        if ' ' in x :        
          nuevo_nombre=x.replace(' ','_')   
          self.__df.rename(columns={x:nuevo_nombre},inplace=True)

    except:
      print('Error al agregar nombres')




  '''Muestra la tabla'''
  def mostrar_tabla (self):
    
     try:
      return(self.__df)
     except:
      print('Error al mostrar el producto con mayor descuento')



  '''Muestra una tabla con los productos mas baratos sin contar el descuento'''
  def productos_mas_baratos_sin_descuento(self):     
     try:
      return(self.__df.groupby(["Product Name","Brand"])[["Listing Price"]].min().sort_values('Listing Price', ascending =  False).tail(10))
     except:
      print('Error al mostrar el producto mas barato sin descuento')



  '''Muestra una tabla con los productos mas caros sin contar el descuento'''
  def productos_mas_caros_sin_descuento(self):
     try:
      return(self.__df.groupby(["Product Name","Brand"])[["Listing Price"]].max().sort_values('Listing Price', ascending =  False).head(10))
     except:
      print('Error al mostrar el producto mas caro sin descuento')

  '''Muestra una tabla con los productos con el mayor descuento'''
  def productos_mayor_descuento(self):
    try:
      return(self.__df.groupby("Product Name")[["Discount"]].median().sort_values('Discount', ascending =  False))
    except:
      print('Error al mostrar el data frame')

    
  '''Muestra una tabla con las ventas de cada marca'''
  def venta_marcas(self):
    try:
      return(self.__df.groupby('Brand')[['Last Visited']].count())
    except:
      print('Error al mostrar el data frame')



      





In [ ]:
# __CREATE DATABASE


class Database:
    '''Create conection and database.db3
    '''
    def __init__(self):
        self.__connection = sqlite3.connect("database.db3")

    def create_table(self):
        '''Create SQL tables in database adidas.db
        '''
        create_adidas_general_table = """ CREATE TABLE IF NOT EXISTS general(Product ID INTEGER PRIMARY KEY,
                                  Product_Name VARCHAR(100) NOT NULL,
                                  Listing_Price FLOAT NOT NULL,
                                  Sale_Price FLOAT NOT NULL,
                                  Discount INT NOT NULL,
                                  Brand    VARCHAR(100) NOT NULL,
                                  Description TEXT NOT NULL,
                                  Last_Visited TIMESTAMP NOT NULL
                                  )
                                  """
        '''Create cursor
        '''
        self.__cursor = self.__connection.cursor()

        self.__cursor.execute(create_adidas_general_table)

    def save_data_to_database(self, df):
        '''Save data into table general
        '''
        df.to_sql('general', self.__connection,
                  if_exists="append", index=False)


if __name__ == '__main__':
    df = Transformacion_DataFrame()
    # df = Transformacion_DataFrame().mostrar()
    df_general = df.mostrar()
    print(df.show_highest_discount())
    print(df.show_most_expensive())
    print(df.show_cheapest())
    print(df.show_more_visited())

    # create database
    db = Database()
    # db.create_table()
    db.save_data_to_database(df_general)

,Product_Name,Product_ID,Listing_Price,Sale_Price,Discount,Brand,Description,Last_Visited
0,Women's adidas Originals NMD_Racer Primeknit S...,AH2430,14999,7499,50,ORIGINALS,Channeling the streamlined look of an '80s rac...,2020-04-13 15:06:14
1,Women's adidas Originals Sleek Shoes,G27341,7599,3799,50,ORIGINALS,"A modern take on adidas sport heritage, tailor...",2020-04-13 15:06:15
2,Women's adidas Swim Puka Slippers,CM0081,999,599,40,CORE / NEO,These adidas Puka slippers for women's come wi...,2020-04-13 15:06:15
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,6999,3499,50,CORE / NEO,"Inspired by modern tech runners, these women's...",2020-04-13 15:06:15
4,Women's adidas Originals Taekwondo Shoes,D98205,7999,3999,50,ORIGINALS,This design is inspired by vintage Taekwondo s...,2020-04-13 15:06:15
...,...,...,...,...,...,...,...,...
2620,Men's adidas Adipower Vector 20 Shoes,EF3503,10999,10999,0,SPORT PERFORMANCE,It's just you and the batsman. As you steam to...,2020-04-13 15:07:59
2621,Men's adidas Cricket Howzat Shoes,EF3505,6999,6999,0,SPORT PERFORMANCE,"Batting, fielding or bowling, these Howzat Spi...",2020-04-13 15:07:59
2622,Men's Cricket Cri Hase Shoes,CM6008,3999,3999,0,SPORT PERFORMANCE,Thiese shoes are a great choice for the amatue...,2020-04-13 15:07:59
2623,Unisex adidas Outdoor Terrex Daroga Water Shoes,BC0980,7999,7999,0,SPORT PERFORMANCE,These shoes fit easily into a travel bag so yo...,2020-04-13 15:07:59
